# 数据预处理

In [10]:
import pandas as pd
from dateutil.parser import parse
import numpy as np
%matplotlib inline

dateparse = lambda dates: pd.datetime.strptime(dates,'%Y-%m-%d')
data = pd.read_csv('../data/all去重改时间.csv' ,dtype={'ORG':object, 'SPEC_CODE':object, 'COMM_MODE':object, 'MANUFACTURER':object,
                                                   'FAULT_QUARTER':object}, date_parser=dateparse)
data.head()

,EQUIP_ID,FAULT_TYPE,detect,inst,fault,saveday,workmonth,INST_DATE,DETECT_DATE,FAULT_DATE,SYNC_ORG_NO,ORG_NO,ORG_NAME,SORT_CODE,SPEC_CODE,COMM_MODE,ARRIVE_BATCH_NO,QTY,MANUFACTURER
0,77,412,,1/4/2013,1/23/2015,,25,2013-1-4,,2015-1-23,64101,6440102,兴庆客户服务分中心,1,113,2,2011000920914723,8000,588
1,275,41201,,1/1/2000,3/12/2015,,182,2000-1-1,,2015-3-12,34101,34401,合肥供电公司,1,138,2,9999138,1326580,1118
2,355,412,5/11/2016,1/4/2013,12/1/2015,-1223,35,2013-1-4,2016-5-11 上午 11:51:0,2015-12-1,64101,6440102,兴庆客户服务分中心,1,113,2,2011000920914723,8000,678
3,415,412,,1/4/2013,2/12/2015,,25,2013-1-4,,2015-2-12,64101,6440101,银川客户服务中心,1,113,2,2011000920914723,8000,588
4,416,412,,1/4/2013,2/12/2015,,25,2013-1-4,,2015-2-12,64101,6440101,银川客户服务中心,1,113,2,2011000920914723,8000,588


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636294 entries, 0 to 1636293
Data columns (total 19 columns):
EQUIP_ID           1636294 non-null int64
FAULT_TYPE         1636294 non-null object
detect             1636294 non-null object
inst               1636294 non-null object
fault              1636294 non-null object
saveday            1636294 non-null object
workmonth          1636294 non-null object
INST_DATE          1636294 non-null object
DETECT_DATE        1636294 non-null object
FAULT_DATE         1636294 non-null object
SYNC_ORG_NO        1636294 non-null int64
ORG_NO             1636294 non-null int64
ORG_NAME           1636294 non-null object
SORT_CODE          1636294 non-null object
SPEC_CODE          1636294 non-null object
COMM_MODE          1636294 non-null object
ARRIVE_BATCH_NO    1636294 non-null object
QTY                1636294 non-null object
MANUFACTURER       1636294 non-null object
dtypes: int64(3), object(16)
memory usage: 237.2+ MB


In [12]:
#转化日期格式
data['FAULT_DATE1'] = pd.to_datetime(data['FAULT_DATE'].str.strip().str.split(' ').str[0])
data['INST_DATE1'] = pd.to_datetime(data['INST_DATE'].str.strip().str.split(' ').str[0])
data['DETECT_DATE1'] = pd.to_datetime(data['DETECT_DATE'].str.strip().str.split(' ').str[0], errors = 'coerce')

In [13]:
data['FAULT_MONTH'] = [x.month for x in data['FAULT_DATE1']]
data['INST_MONTH'] = [x.month for x in data['INST_DATE1']]

In [14]:
data['INST_MONTH'] = pd.to_numeric(data['INST_MONTH'], downcast='signed', errors = 'coerce')

In [15]:
from datetime import timedelta
from IPython.core.debugger import Tracer
print sum(data['FAULT_DATE1'].isnull()), sum(data['INST_DATE1'].isnull())
data['work_month'] = data['FAULT_DATE1'] - data['INST_DATE1']
data['work_month'] = [x.days/30 if not pd.isnull(x) else np.nan for x in data['work_month']]

data['save_day'] = data['INST_DATE1'] - data['DETECT_DATE1']
data['save_day'] = [x.days if not pd.isnull(x) else np.nan for x in data['save_day']]

month_mean = data['work_month'].mean()
month_std = data['work_month'].std()
count_nan_month = data['work_month'].isnull().sum()
count_0_month = len(data['work_month'][data['work_month']<0])
#随机取正态分布进行补充缺失值
rand_1 = np.random.randint(month_mean - month_std, month_mean + month_std, size = count_nan_month + count_0_month)
data['work_month'][(data['work_month'].isnull()) | (data['work_month']<0)] = rand_1

day_mean = data['save_day'].mean()
day_std = data['save_day'].std()
count_nan_day = data['save_day'].isnull().sum()
count_0_day = len(data['save_day'][data['save_day']<0])
#随机取正态分布进行补充缺失值
rand_2 = np.random.randint(day_mean - day_std, day_mean + day_std, size = count_nan_day + count_0_day)

data['save_day'][(data['save_day'].isnull()) | (data['save_day']<0)] = rand_2

0 56590


/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
#筛选数据
data = data[data['SORT_CODE']==10] #选出SORT_CODE 为10 的智能表
data['FAULT_TYPE_1'] = [x[0] for x in data['FAULT_TYPE'].values.astype('str')]
data['FAULT_TYPE'] = [x[:3] for x in data['FAULT_TYPE'].values.astype('str')]
data = data[data['FAULT_TYPE_1']=='4']
data = data[(data['FAULT_TYPE']!='412')&(data['FAULT_TYPE']!='4')]
data['FAULT_TYPE'].value_counts()
#data['ORG_NO'] = [x[:5] for x in data['ORG_NO'].values.astype('str')]

409    127096
411     76257
407     71985
408     57424
404     54128
401     35836
410     17851
402     17329
405      4914
406      3643
403      2377
Name: FAULT_TYPE, dtype: int64

In [17]:
#删除无用数据
data.drop(['SORT_CODE', 'FAULT_DATE', 'INST_DATE','DETECT_DATE','detect', 'inst', 'fault', 'workmonth', 'QTY','saveday'], 
         axis=1, inplace=True)

In [18]:
data.to_csv('../data/all_2.csv', index=False)

## 